<a href="https://colab.research.google.com/github/Howard-hth/capstone/blob/main/dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [88]:
!pip install plotly dash

In [89]:
# Import required libraries
import dash
from dash import dcc, html, Input, Output
import pandas as pd
import plotly.express as px

# Read the airline data into pandas dataframe
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"
spacex_df = pd.read_csv(url)
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Get unique launch sites and add 'ALL SITES'
launch_sites = spacex_df['Launch Site'].unique().tolist()
launch_sites.insert(0, 'ALL SITES')

In [90]:
spacex_df.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [92]:
# Initialize the Dash app
app = dash.Dash(__name__)

# Layout of the app
app.layout = html.Div([
    dcc.Dropdown(
        id='site-dropdown',
        options=[{'label': site, 'value': site} for site in launch_sites],
        value='ALL SITES',  # Default value
        clearable=False,
    ),
    html.Br(),
    html.P("Payload range (Kg):"),
    dcc.RangeSlider(
        id='payload-slider',
        min=0,
        max=10000,
        step=100,
        value=[min_payload, max_payload],  # Default range selection
        marks={i: str(i) for i in range(0, 10001, 1000)},  # Marks on the slider
        allowCross=False,
    ),
    html.Br(),
    html.Div(dcc.Graph(id='success-pie-chart')),  # Pie chart
    html.Br(),
    html.Div(dcc.Graph(id='success-payload-scatter-chart')),  # Scatter chart
])

# Callback for the pie chart
@app.callback(
    Output('success-pie-chart', 'figure'),
    Input('site-dropdown', 'value')
)
def update_pie_chart(selected_site):
    if selected_site == 'ALL SITES':
        success_counts = spacex_df['class'].value_counts()
        labels = ['Failed', 'Successful']
        values = [len(spacex_df) - success_counts.get(1, 0), success_counts.get(1, 0)]
    else:
        site_data = spacex_df[spacex_df['Launch Site'] == selected_site]
        success_counts = site_data['class'].value_counts()
        labels = ['Failed', 'Successful']
        values = [len(site_data) - success_counts.get(1, 0), success_counts.get(1, 0)]

    # Create pie chart
    fig = px.pie(values=values, names=labels, title=f'Success vs. Failed Launches for {selected_site}')
    return fig

# Callback for the scatter chart
@app.callback(
    Output('success-payload-scatter-chart', 'figure'),
    Input('site-dropdown', 'value'),
    Input('payload-slider', 'value')
)
def update_scatter_chart(selected_site, payload_range):
    # Filter data based on the selected site
    if selected_site == 'ALL SITES':
        filtered_df = spacex_df
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == selected_site]

    # Further filter based on the payload range
    filtered_df = filtered_df[(filtered_df['Payload Mass (kg)'] >= payload_range[0]) & (filtered_df['Payload Mass (kg)'] <= payload_range[1])]

    # Check if filtered data is empty
    if filtered_df.empty:
        fig = px.scatter(title=f'No data for {selected_site} in the selected payload range.')
    else:
        fig = px.scatter(
            filtered_df,
            x='Payload Mass (kg)',
            y='class',
            title=f'Payload vs Launch Success for {selected_site}',
            labels={'Payload Mass (kg)': 'Payload (Kg)', 'class': 'Launch Success'},
            hover_data=['Launch Site']
        )
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

<IPython.core.display.Javascript object>